In [ ]:
import pandas as pd
import numpy as np
import joblib
from tqdm.auto import tqdm, trange
import nltk
nltk.download('punkt')
nltk.download("stopwords")

In [ ]:
data = pd.read_csv('lenta-ru-news.csv')

In [ ]:
data.shape

In [ ]:
#data = data[:800000]

In [ ]:
data.head(5)

In [ ]:
print(data['tags'].unique(), len(data['tags'].unique()))

In [ ]:
print(data['topic'].unique(), len(data['topic'].unique()))

## Выборка данных

In [ ]:
tags = [
    'Кино', 'Преступность', 'Общество', 'Происшествия',
    'Искусство', 'Бизнес', 'Техника', 'ТВ и радио', 'Политика',
    'Пресса', 'Музыка', 'Люди', 'Звери', 'Игры', 'Культура'
    'Гаджеты', 'Наука', 'Еда', 'Рынки', 'Деньги', 'Интернет',
    'Театр', 'Реклама', 'Космос', 'Бокс и ММА', 'Футбол', 'Книги',
    'Достижения', 'Coцсети', 'События', 'Софт', 'Квартира', 'Город', 'Дача',
    'Офис', 'Оружие', 'Криминал',  'Мемы', 'Инновации', 'Хоккей',
    'Стиль', 'Инструменты', 'История', 'Туризм', 'Экология',
    'Фотография', 'Авто', 'Жизнь', 'Киберпреступность', 'Экономика',
]
news_in_cat_count = 2200

In [ ]:
df_res = pd.DataFrame()

for tag in tqdm(tags):
    df_topic = data[data['tags'] == tag][:news_in_cat_count]
    df_res = df_res.append(df_topic, ignore_index=True)

In [ ]:
df_res.shape

## Предобработка

In [ ]:
import string
def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

import re
def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

In [ ]:

from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
mystem = Mystem()

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...'])


def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [
        token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text


In [ ]:
preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
data['preproccessed'] = list(map(preproccessing, df_res['text']))

In [ ]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df_res['text'])]

In [ ]:
len(prep_text)
prep_text[0]

In [ ]:
df_res['text_prep'] = prep_text

In [ ]:
df_res.head(1)

### Стемминг

In [ ]:
from nltk.stem.snowball import SnowballStemmer 
stemmer = SnowballStemmer("russian") 

In [ ]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])

In [ ]:
text = df_res['text_prep'][0]

In [ ]:
from nltk import word_tokenize

stemmed_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
    text = " ".join(stemmed_tokens)
    stemmed_texts_list.append(text)

df_res['text_stem'] = stemmed_texts_list


In [ ]:
from nltk import word_tokenize
def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

In [ ]:
from nltk import word_tokenize

sw_texts_list = []
for text in tqdm(df_res['text_prep']):
    tokens = word_tokenize(text)    
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    text = " ".join(tokens)
    sw_texts_list.append(text)

df_res['text_sw'] = sw_texts_list

In [ ]:
df_res['text_sw'][0]

In [ ]:
df_res.to_csv('lenta_stemmed.csv')

In [ ]:
df_res['text_stem'][0]

In [ ]:
df_res.head(1)


## Лемматизация

In [ ]:
df_res = pd.read_csv('lenta_stemmed.csv')

In [ ]:
lemm_texts_list = []
for text in tqdm(df_res['text_sw']):
    #print(text)
    try:
        text_lem = mystem.lemmatize(text)
        tokens = [token for token in text_lem if token != ' ' and token not in russian_stopwords]
        text = " ".join(tokens)
        lemm_texts_list.append(text)
    except Exception as e:
        print(e)
    
df_res['text_lemm'] = lemm_texts_list

In [ ]:
def lemmatize_text(text):
    text_lem = mystem.lemmatize(text)
    tokens = [token for token in text_lem if token != ' ']
    return " ".join(tokens)

In [ ]:
df_res.to_csv('lemm.csv')

In [ ]:
df_res = pd.read_csv('lemm.csv', encoding = 'utf-8')

In [ ]:
df_res.head()

In [ ]:
df_res['text_lemm'][0]

In [ ]:
#X = df_res['text_lemm']
X = df_res['text_sw']
y = df_res['tags']

In [ ]:
type(df_res['text_lemm'][0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [ ]:
my_tags = df_res['tags'].unique()
my_tags

In [ ]:
type(X_train[1])
df['Review'].values.astype('U')

## Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X_train, y_train)

In [ ]:
joblib.dump(nb,'native_bayes.pkl')

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
print(X_test[3000], y_test[3000], y_pred[3000])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

## Linear Support Vector Machine


In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
%%time
sgd.fit(X_train, y_train)

In [ ]:
joblib.dump(sgd,'sgd.pkl')

In [ ]:
%%time
y_pred = sgd.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [ ]:
%%time
logreg.fit(X_train, y_train)

In [ ]:
joblib.dump(logreg,'logistic_regression.pkl')

In [ ]:
%%time
y_pred = logreg.predict(X_test)

In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

### Проверка на примерах новостей с сайта

In [ ]:
econ_text = '''
Доллар США считается символом надежности, безопасности и экономического процветания. 
Он занимает неоспоримое доминирующее положение в международной финансовой системе с середины XX века и производит 
впечатление непобедимого титана. Однако эра господства доллара как основной мировой резервной валюты медленно 
подходит к концу. Крупнейшие банки предрекают ему резкий спад уже в следующем году, а известный экономист 
Стивен Роуч уверен, что американская валюта может обесцениться на треть. 
Причинами обвала станут сокращение сбережений населения, рост государственного долга США и усиление Китая. 
Закат долларового диктата — в материале «Ленты.ру».
Непомерные привилегии

Успех американской экономики в XX веке был во многом обусловлен доминирующей ролью доллара. В свою очередь достижение этой роли стало результатом политического и военного превосходства, которое США приобрели после Первой мировой войны. До сих пор позиция доллара в мире финансов представляет собой главную основу процветания США. Однако в 2002 году наметилась долгосрочная тенденция к ослаблению американской валюты, которая наблюдается и по сей день.

Действительно, последнее время доллар чувствует себя не очень хорошо. В частности, июль оказался очень сложным месяцем для американской валюты, которая обновила многомесячные, а в некоторых случаях и многолетние минимумы. Всего за месяц доллар подешевел на шесть процентов против фунта, на пять — по отношению к евро, на четыре — против швейцарского франка и австралийского доллара. Вдобавок валютные аналитики вполне допускают, что июль может оказаться для доллара худшим месяцем с точки зрения месячной динамики за последние десять лет.

'''

In [ ]:
econ_text = remove_multiple_spaces(remove_numbers(remove_punctuation(econ_text.lower())))
econ_text = remove_stop_words(econ_text)

In [ ]:
%%time
econ_text = lemmatize_text(econ_text)

In [ ]:
econ_text

In [ ]:
ect_pred1 = nb.predict([econ_text])
ect_pred2 = sgd.predict([econ_text])
ect_pred3 = logreg.predict([econ_text])
print(ect_pred1, ect_pred2, ect_pred3)


In [ ]:
tech_text = '''
Ученые Университета Аризоны в США открыли неизвестный принцип эволюции, согласно которому естественный отбор не может благоприятствовать сразу нескольким полезным мутациям, повышающим приспособленность организма к окружающей среде. Вместо этого он фокусируется только на том, что достаточно для выживания существа, и игнорирует другие возможности для улучшения. Об этом сообщается в статье в журнале Proceedings of the National Academy of Sciences.
Исследователи спровоцировали несколько мутаций, которые «ломали» механизм трансляции у шести различных штаммов бактерии кишечной палочки Escherichia coli. Трансляция — это процесс, при котором заложенная в генах информация о нуклеотидной последовательности белков переносится к рибосомам и участвует в белковом синтезе. Известно, что более чем за 3,5 миллиарда лет эволюции механизм трансляции не претерпел существенных изменений.
Ряд нефатальных мутаций, затрагивающих функционирование трансляционного процесса, должен с течением времени быть исправлен эволюцией. Для этого ученые заставили штаммы бактерий с различными изменениями конкурировать друг с другом, благодаря чему осуществлялся естественный отбор. Со временем внутри популяции накапливались полезные мутации, которые могли быть сохранены и использованы для улучшения трансляции.'''

In [ ]:
tech_text = remove_multiple_spaces(remove_numbers(remove_punctuation(tech_text.lower())))
tech_text = remove_stop_words(tech_text)
#tech_text = lemmatize_text(tech_text)

In [ ]:
tech_pred1 = nb.predict([tech_text])
tech_pred2 = sgd.predict([tech_text])
tech_pred3 = logreg.predict([tech_text])
print(tech_pred1, tech_pred2, tech_pred3)


In [ ]:
travel_text = '''
Российские туристы с 1 августа могут беспрепятственно въезжать в Абхазию, а граждане этой страны — в Россию, соответствующее распоряжение об открытии границ между странами подписал премьер-министр России Михаил Мишустин.
Границу уже свободно пересекают пешеходы и транспорт, все введенные ранее на фоне пандемии коронавируса ограничения сняты.
При этом о своем намерении российские власти сообщили еще в пятницу, 31 июля — решение было принято по итогам заседания оперативного штаба по борьбе с распространением инфекции на основании заключений Роспотребнадзора и Минздрава России по оценке санитарно-эпидемиологической обстановки в стране, а также определения готовности объектов абхазской инфраструктуры к приему туристов.
'''

In [ ]:
travel_text = remove_multiple_spaces(remove_numbers(remove_punctuation(travel_text.lower())))
travel_text = remove_stop_words(travel_text)
#travel_text = lemmatize_text(travel_text)

In [ ]:
travel_pred1 = nb.predict([travel_text])
travel_pred2 = sgd.predict([travel_text])
travel_pred3 = logreg.predict([travel_text])
print(travel_pred1, travel_pred2, travel_pred3)
